In [1]:
import pandas as pd, sqlite3

In [2]:
conn = sqlite3.connect('nba.sqlite/nba.sqlite')
curs = conn.cursor

### Messing around

In [22]:
pd.read_sql('''SELECT count(*)
                FROM play_by_play
                WHERE score is not null
                and scoremargin != 'TIE'
                and SUBSTR(score,1,1) != '0'
                and period > 2;''', conn)

,count(*)
0,1761976


#### casting scoremargin

In [ ]:
tatum = pd.read_sql('''SELECT game_id, period, pctimestring, score, CAST(scoremargin AS INTEGER) as scoremargin
                FROM play_by_play
                WHERE score is not null
                and scoremargin != 'TIE'
                and SUBSTR(score,1,1) != '0'
                and period > 4
                LIMIT 10;''', conn)
tatum.dtypes

game_id         object
period           int64
pctimestring    object
score           object
scoremargin      int64
dtype: object

### Margin excl

In [ ]:
zingis = pd.read_sql('''WITH int_margin AS(
                     SELECT game_id, period, pctimestring, score, CAST(scoremargin AS INTEGER) as scoremargin
                        FROM play_by_play
                        WHERE score is not null
                        and scoremargin != 'TIE'
                        and SUBSTR(score,1,1) != '0'
                        and period > 4)
                     SELECT * FROM int_margin
                     WHERE scoremargin < -10
                        OR scoremargin > 10
                     ;''', conn)
zingis

,game_id,period,pctimestring,score,scoremargin
0,0029600086,6,0:20,126 - 115,-11
1,0029600086,6,0:00,126 - 115,-11
2,0029600217,5,1:32,94 - 83,-11
3,0029600217,5,1:32,95 - 83,-12
4,0029600217,5,1:25,96 - 85,-11
...,...,...,...,...,...
242,0022200993,5,0:49,121 - 132,11
243,0022200993,5,0:31,123 - 134,11
244,0022201040,5,1:12,134 - 123,-11
245,0022201040,5,0:38,137 - 125,-12


### NET COLUMN

In [49]:
play_by_play = pd.read_sql(''' WITH int_margin AS(
                     SELECT game_id, period, pctimestring, score, CAST(scoremargin AS INTEGER) as scoremargin
                        FROM play_by_play
                        WHERE score is not null
                        and scoremargin != 'TIE'
                        and SUBSTR(score,1,1) != '0'
                        and period > 3)
                   
                     SELECT *, (CASE WHEN scoremargin < 0 THEN scoremargin*(-1) ELSE scoremargin END) as grossmargin
                      FROM int_margin
                      WHERE scoremargin < -10
                        OR scoremargin > 10
                      ORDER BY period desc, pctimestring asc, grossmargin desc
                      ;''', conn)
print(play_by_play['grossmargin'].value_counts())
play_by_play.head()

grossmargin
11    37145
12    34159
13    31399
14    28944
15    26609
      ...  
67        2
74        2
76        2
78        2
72        1
Name: count, Length: 66, dtype: int64


,game_id,period,pctimestring,score,scoremargin,grossmargin
0,0020800979,7,0:00,129 - 140,11,11
1,0020000923,7,0:16,116 - 127,11,11
2,0020000923,7,0:26,114 - 125,11,11
3,0020800979,7,0:35,129 - 140,11,11
4,0021400390,7,1:57,125 - 114,-11,11


### Split score and find doubles

In [67]:
def find_doubles(play_by_play):
    # split score on '-' and return two integer columns for visitor and home
    play_by_play[['visitor','home']] = play_by_play['score'].str.split('-',n=1, expand = True).apply(pd.to_numeric)

    # find all records where grossmargin is >= home or visitor score (i.e. more than doubled)
    doubles = play_by_play[(play_by_play['grossmargin'] >= play_by_play['visitor']) | (play_by_play['grossmargin'] >= play_by_play['home'])]
    doubles = doubles.drop_duplicates('game_id')
    return doubles
find_doubles(play_by_play)

,game_id,period,pctimestring,score,scoremargin,grossmargin,visitor,home
248,0012100048,4,0:00,127 - 59,-68,68,127,59
249,0029700812,4,0:00,59 - 124,65,65,59,124
48897,0011300020,4,10:06,40 - 82,42,42,40,82
55687,0029900305,4,10:21,38 - 80,42,42,38,80
68474,0020100073,4,10:50,48 - 101,53,53,48,101
78504,0012200069,4,11:14,46 - 22,-24,24,46,22
79248,0021000560,4,11:16,43 - 92,49,49,43,92
83143,0029900763,4,11:25,37 - 77,40,40,37,77
88616,0021700810,4,11:40,100 - 50,-50,50,100,50
89107,0021701072,4,11:41,57 - 114,57,57,57,114


## Playoffs

#### playoff games

In [60]:
playoffs = pd.read_sql('''SELECT game_id, SUBSTR(game_date,1,10) as game_date, team_abbreviation_home, team_abbreviation_away, season_type 
                            FROM game
                            WHERE season_type = 'Playoffs';''', conn)
playoffs

,game_id,game_date,team_abbreviation_home,team_abbreviation_away,season_type
0,0044601021,1947-04-02,CLR,NYK,Playoffs
1,0044600221,1947-04-02,WAS,CHS,Playoffs
2,0044601011,1947-04-02,PHW,BOM,Playoffs
3,0044600222,1947-04-03,WAS,CHS,Playoffs
4,0044601012,1947-04-05,BOM,PHW,Playoffs
...,...,...,...,...,...
3837,0042200401,2023-06-01,DEN,MIA,Playoffs
3838,0042200402,2023-06-04,DEN,MIA,Playoffs
3839,0042200403,2023-06-07,MIA,DEN,Playoffs
3840,0042200404,2023-06-09,MIA,DEN,Playoffs


#### full query

In [69]:
playoff_by_play = pd.read_sql(''' WITH int_margin AS(
                     SELECT game_id, period, pctimestring, score, CAST(scoremargin AS INTEGER) as scoremargin
                        FROM play_by_play
                        WHERE score is not null
                        and scoremargin != 'TIE'
                        and SUBSTR(score,1,1) != '0'
                        and period > 2),
                    playoffs as (SELECT game_id, SUBSTR(game_date,1,10) as game_date, team_abbreviation_home, team_abbreviation_away, season_type 
                            FROM game
                            WHERE season_type = 'Playoffs')
                            
                   
                     SELECT playoffs.game_id, game_date, team_abbreviation_away, team_abbreviation_home,
                                period, pctimestring, score, (CASE WHEN scoremargin < 0 THEN scoremargin*(-1) ELSE scoremargin END) as grossmargin
                      FROM playoffs
                        JOIN int_margin ON playoffs.game_id = int_margin.game_id

                      WHERE scoremargin < -10
                        OR scoremargin > 10
                      ORDER BY period desc, pctimestring asc, grossmargin desc
                      ;''', conn)
playoff_by_play

,game_id,game_date,team_abbreviation_away,team_abbreviation_home,period,pctimestring,score,grossmargin
0,0041400123,2015-04-23,CHI,MIL,6,1:15,113 - 101,12
1,0041400123,2015-04-23,CHI,MIL,6,1:15,112 - 101,11
2,0041900173,2020-08-22,HOU,OKC,5,0:00,107 - 119,12
3,0049700053,1998-05-10,IND,NYK,5,0:00,118 - 107,11
4,0040400154,2005-05-02,SAS,DEN,5,0:00,126 - 115,11
...,...,...,...,...,...,...,...,...
42080,0040700311,2008-05-21,SAS,LAL,3,9:59,57 - 43,14
42081,0042000215,2021-06-15,MIL,BKN,3,9:59,64 - 51,13
42082,0042000162,2021-05-24,POR,DEN,3,9:59,64 - 76,12
42083,0041600112,2017-04-17,IND,CLE,3,9:59,61 - 72,11


In [70]:
find_doubles(playoff_by_play)

,game_id,game_date,team_abbreviation_away,team_abbreviation_home,period,pctimestring,score,grossmargin,visitor,home
10496,0040800154,2009-04-27,DEN,NOH,4,3:06,115 - 57,58,115,57
25524,0041800125,2019-04-23,BKN,PHI,3,10:04,31 - 68,37,31,68
25635,0042100227,2022-05-15,DAL,PHX,3,10:10,65 - 27,38,65,27
25936,0040000081,2001-06-01,PHI,MIL,3,10:24,33 - 66,33,33,66
26052,0041600302,2017-05-19,CLE,BOS,3,10:29,77 - 35,42,77,35
26431,0049600012,1997-04-27,ORL,MIA,3,10:50,31 - 62,31,31,62
26930,0041400126,2015-04-30,CHI,MIL,3,11:16,66 - 33,33,66,33
27436,0041500202,2016-05-04,ATL,CLE,3,11:44,38 - 76,38,38,76
31814,0040700107,2008-05-04,ATL,BOS,3,3:05,34 - 70,36,34,70
40994,0041100111,2012-04-28,NYK,MIA,3,9:11,31 - 62,31,31,62
